# KSI monitoring: zpětná vazba na úlohy
V tomto souboru se nachází statistika kvantitativní zpětné vazby řešitelů ke konkrétním úlohám.

In [1]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns

import util
from util.year import year as current_year
from db import session
import model
from datetime import datetime
import json

pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = [8, 6]
print(datetime.now())

2019-08-02 19:23:32.471806


In [3]:
all_waves = session.query(model.Wave).\
    filter(model.Wave.year == current_year.id).\
    order_by(model.Wave.index).all()

In [4]:
Feedback = namedtuple(
    'Feedback',
    ['user', 'task', 'explained', 'interesting', 'difficult']
)

feedbacks_raw = session.query(
    model.Feedback,
    model.Task,
).\
    join(model.Feedback.r_task).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Feedback.r_user).\
    filter(model.User.role == 'participant').\
    order_by(model.Task.id).all()

feedbacks = []
for f, task in feedbacks_raw:
    data = {
        item['id']: item['answer']
        for item in json.loads(f.content)
    }
    feedbacks.append(Feedback(
        f.user,
        task,
        data['explained'] if 'explained' in data else None,
        data['interesting'] if 'interesting' in data else None,
        data['difficult'] if 'difficult' in data else None,
    ))

In [5]:
large_tasks = util.task.large_tasks().all()

In [8]:
def show_wave_feedbacks(wave, max_feedbacks_count):
    feedbacks_in_wave = [ 
        f
        for f in feedbacks
        if f.task.wave == wave.id
    ]
    feedbacks_in_wave.sort(
        key=lambda x: (x.task in large_tasks, x.task.id)
    )
    
    df = pd.DataFrame(feedbacks_in_wave)
    if df.empty:
        return
    
    grouped = df.groupby('task')
    per_task = pd.DataFrame(OrderedDict((
        ('Explained Count', grouped.explained.count()),
        ('Interesting Count', grouped.interesting.count()),
        ('Difficult Count', grouped.difficult.count()),
        ('Explained Mean', grouped.explained.mean()),    
        ('Interesting Mean', grouped.interesting.mean()),    
        ('Difficult Mean', grouped.difficult.mean()),
    )))
    
    s = per_task.style

    s.format({
        'Explained Mean': '{:.2f}'.format,
        'Interesting Mean': '{:.2f}'.format,
        'Difficult Mean': '{:.2f}'.format,
    })
    
    s.bar(subset=[
        'Explained Count', 'Interesting Count', 'Difficult Count'
    ], color='#5fd65f', vmin=0, vmax=max_feedbacks_count)
    
    s.background_gradient(subset=[
        'Explained Mean',
        'Interesting Mean',
    ], cmap=sns.light_palette("green", as_cmap=True))
    
    s.background_gradient(subset=[
        'Difficult Mean',
    ], cmap=sns.light_palette("red", as_cmap=True))    
    
    display(Markdown('## {name}'.format(name=wave.caption)))
    display(s)

counts = {}
for feedback in feedbacks:
    counts[feedback.task] = counts.get(feedback.task, 0) + 1

max_feedbacks_count = max(counts.values(), default=100)

for wave in all_waves:
    show_wave_feedbacks(wave, max_feedbacks_count)